In [ ]:
import sys
!{sys.executable} -m pip install pandas matplotlib colour numpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.close("all")
dir_logs = "./imports/"
tick_file = "strategy-tick.csv"
update_file = "state-update.csv"

In [ ]:
def get_timestamp_payload(path):
    raw_data = pd.read_csv(path)
    data = raw_data[["jsonPayload.message", "timestamp"]]
    data.columns = ["payload", "timestamp"]
    data = data.assign(payload_timestamp = data["payload"].str.extract(r'(.+?(?=\shopr:cover-traffic\s))'))
    data["payload"] = data["payload"].str.extract(r'hopr:cover-traffic\s(.*)')
    return data

In [ ]:
def parse_strategy_tick(data):
    data = data.assign(tick_timestamp = data["payload"].str.extract(r'strategy tick:\s?(\d*)\s', expand=False))
    data = data.assign(balance = data["payload"].str.extract(r'balance:\s?(\d*)\s'))
    data = data.assign(rounded_balance = data["balance"].astype(float).div(1e18))
    data = data.assign(open_to_node = data["payload"].str.extract(r'open:\s?([\w\,]*)\s'))
    data = data.assign(close_to_node = data["payload"].str.extract(r'close:\s?([\w\,]*)\s?'))
    data["tick_timestamp"] =  pd.to_datetime(data["tick_timestamp"].apply(pd.to_numeric), unit="ms")
    return data[["timestamp", "payload_timestamp", "tick_timestamp", "balance", "rounded_balance", "open_to_node", "close_to_node"]]

In [ ]:
tick_logs = parse_strategy_tick(get_timestamp_payload(dir_logs + tick_file))

In [ ]:
tick_logs

In [ ]:
tick_logs.dtypes

In [ ]:
plt.figure()
tick_logs.plot(x="tick_timestamp", y="rounded_balance", kind="bar")